# Manufacturing Operation Data Analytics

## Objective
Analyze manufacturing operation data to discover optimizations, hidden correlations, and improve decision-making.

## Key Questions
1. Should we use market activity (WEAK/GROWING/STRONG/RESTRICTED) in decisions?
2. Are our position sizing multipliers optimal?
3. What hidden correlations exist in our data?
4. Can we predict supply level changes?
5. What other optimizations can we find?

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
from scipy import stats
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = [12, 6]

print('Libraries loaded successfully!')

## 1. Data Extraction from PostgreSQL

In [ ]:
# Database connection
# Update these credentials to match your .env
DB_HOST = '127.0.0.1'
DB_PORT = '5432'
DB_NAME = 'spacetraders'
DB_USER = 'spacetraders'
DB_PASSWORD = 'dev_password'
PLAYER_ID = 12  # Update to your player ID

engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')

print(f'Connected to {DB_NAME} database')

In [ ]:
# Load market price history
query_price_history = f"""
SELECT 
    waypoint_symbol, good_symbol,
    purchase_price, sell_price, 
    supply, activity, trade_volume,
    recorded_at
FROM market_price_history
WHERE player_id = {PLAYER_ID}
ORDER BY recorded_at DESC
"""
df_prices = pd.read_sql(query_price_history, engine)
df_prices['recorded_at'] = pd.to_datetime(df_prices['recorded_at'])
print(f'Market price history: {len(df_prices):,} records')
print(f'Date range: {df_prices["recorded_at"].min()} to {df_prices["recorded_at"].max()}')
df_prices.head()

In [ ]:
# Load transactions (ledger)
query_transactions = f"""
SELECT 
    id, timestamp, transaction_type, category, 
    amount, balance_before, balance_after,
    description, metadata, operation_type,
    related_entity_type, related_entity_id,
    created_at
FROM transactions
WHERE player_id = {PLAYER_ID}
ORDER BY created_at DESC
"""
df_transactions = pd.read_sql(query_transactions, engine)
df_transactions['created_at'] = pd.to_datetime(df_transactions['created_at'])
print(f'Transactions: {len(df_transactions):,} records')
df_transactions.head()

In [ ]:
# Load manufacturing tasks
query_tasks = f"""
SELECT 
    id, pipeline_id, task_type, status, good, quantity, actual_quantity,
    source_market, target_market, factory_symbol,
    assigned_ship, priority, retry_count, max_retries,
    total_cost, total_revenue, error_message,
    created_at, ready_at, started_at, completed_at
FROM manufacturing_tasks
WHERE player_id = {PLAYER_ID}
ORDER BY created_at DESC
"""
df_tasks = pd.read_sql(query_tasks, engine)
for col in ['created_at', 'ready_at', 'started_at', 'completed_at']:
    df_tasks[col] = pd.to_datetime(df_tasks[col])
print(f'Manufacturing tasks: {len(df_tasks):,} records')
df_tasks.head()

In [ ]:
# Load manufacturing pipelines
query_pipelines = f"""
SELECT 
    id, sequence_number, pipeline_type, product_good, sell_market,
    expected_price, status, total_cost, total_revenue, net_profit,
    error_message, created_at, started_at, completed_at
FROM manufacturing_pipelines
WHERE player_id = {PLAYER_ID}
ORDER BY created_at DESC
"""
df_pipelines = pd.read_sql(query_pipelines, engine)
for col in ['created_at', 'started_at', 'completed_at']:
    df_pipelines[col] = pd.to_datetime(df_pipelines[col])
print(f'Manufacturing pipelines: {len(df_pipelines):,} records')
df_pipelines.head()

In [ ]:
# Load current market data
query_market = f"""
SELECT 
    waypoint_symbol, good_symbol, supply, activity,
    purchase_price, sell_price, trade_volume, trade_type,
    last_updated
FROM market_data
WHERE player_id = {PLAYER_ID}
"""
df_market = pd.read_sql(query_market, engine)
df_market['last_updated'] = pd.to_datetime(df_market['last_updated'])
print(f'Current market data: {len(df_market):,} records')
df_market.head()

In [ ]:
# Load container logs
query_logs = f"""
SELECT 
    container_id, timestamp, level, message, metadata
FROM container_logs
WHERE player_id = {PLAYER_ID}
ORDER BY timestamp DESC
LIMIT 10000
"""
df_logs = pd.read_sql(query_logs, engine)
df_logs['timestamp'] = pd.to_datetime(df_logs['timestamp'])
print(f'Container logs: {len(df_logs):,} records')
df_logs.head()

## 2. Exploratory Data Analysis (EDA)

In [ ]:
# Summary statistics for price history
print('=== Price History Summary ===')
print(f'Unique waypoints: {df_prices["waypoint_symbol"].nunique()}')
print(f'Unique goods: {df_prices["good_symbol"].nunique()}')
print(f'\nSupply distribution:')
print(df_prices['supply'].value_counts())
print(f'\nActivity distribution:')
print(df_prices['activity'].value_counts())

In [ ]:
# Visualize supply and activity distributions
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Supply distribution
supply_order = ['SCARCE', 'LIMITED', 'MODERATE', 'HIGH', 'ABUNDANT']
supply_counts = df_prices['supply'].value_counts().reindex(supply_order).fillna(0)
axes[0].bar(supply_counts.index, supply_counts.values, color='steelblue')
axes[0].set_title('Supply Level Distribution', fontsize=14)
axes[0].set_xlabel('Supply Level')
axes[0].set_ylabel('Count')
axes[0].tick_params(axis='x', rotation=45)

# Activity distribution
activity_order = ['WEAK', 'RESTRICTED', 'GROWING', 'STRONG']
activity_counts = df_prices['activity'].value_counts().reindex(activity_order).fillna(0)
axes[1].bar(activity_counts.index, activity_counts.values, color='coral')
axes[1].set_title('Activity Level Distribution', fontsize=14)
axes[1].set_xlabel('Activity Level')
axes[1].set_ylabel('Count')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('outputs/figures/supply_activity_distribution.png', dpi=150)
plt.show()

In [ ]:
# Transaction summary
print('=== Transaction Summary ===')
print(f'\nBy transaction type:')
print(df_transactions.groupby('transaction_type')['amount'].agg(['count', 'sum', 'mean']))
print(f'\nBy category:')
print(df_transactions.groupby('category')['amount'].agg(['count', 'sum', 'mean']))
print(f'\nBy operation type:')
print(df_transactions.groupby('operation_type')['amount'].agg(['count', 'sum', 'mean']))

In [ ]:
# Task summary
print('=== Task Summary ===')
print(f'\nBy task type:')
print(df_tasks.groupby('task_type')[['total_cost', 'total_revenue']].agg(['count', 'sum', 'mean']))
print(f'\nBy status:')
print(df_tasks['status'].value_counts())

In [ ]:
# Pipeline profitability summary
print('=== Pipeline Summary ===')
completed_pipelines = df_pipelines[df_pipelines['status'] == 'COMPLETED']
print(f'Completed pipelines: {len(completed_pipelines)}')
if len(completed_pipelines) > 0:
    print(f'\nProfitability by product:')
    print(completed_pipelines.groupby('product_good')[['total_cost', 'total_revenue', 'net_profit']].agg(['count', 'sum', 'mean']))

## 3. Market Activity Analysis (PRIORITY - Currently Unused!)

In [ ]:
# H1: Does activity level correlate with price changes?
print('=== H1: Market Activity vs Price Changes ===')

# Calculate price changes for each waypoint-good combination
df_prices_sorted = df_prices.sort_values(['waypoint_symbol', 'good_symbol', 'recorded_at'])
df_prices_sorted['price_change'] = df_prices_sorted.groupby(['waypoint_symbol', 'good_symbol'])['purchase_price'].diff()
df_prices_sorted['price_change_pct'] = df_prices_sorted.groupby(['waypoint_symbol', 'good_symbol'])['purchase_price'].pct_change() * 100

# Analyze price changes by activity level
activity_price_stats = df_prices_sorted.groupby('activity').agg({
    'price_change': ['mean', 'std', 'count'],
    'price_change_pct': ['mean', 'std'],
    'purchase_price': 'mean'
}).round(2)
print('\nPrice change statistics by activity level:')
print(activity_price_stats)

In [ ]:
# Visualize price volatility by activity level
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Box plot of price changes by activity
activity_data = df_prices_sorted[df_prices_sorted['activity'].notna()]
if len(activity_data) > 0:
    activity_order = ['WEAK', 'RESTRICTED', 'GROWING', 'STRONG']
    available_activities = [a for a in activity_order if a in activity_data['activity'].unique()]
    
    sns.boxplot(data=activity_data, x='activity', y='price_change_pct', 
                order=available_activities, ax=axes[0])
    axes[0].set_title('Price Change % by Activity Level', fontsize=14)
    axes[0].set_xlabel('Activity Level')
    axes[0].set_ylabel('Price Change %')
    axes[0].set_ylim(-50, 50)  # Clip outliers for visibility

    # Mean price by activity
    mean_prices = activity_data.groupby('activity')['purchase_price'].mean().reindex(available_activities)
    axes[1].bar(mean_prices.index, mean_prices.values, color='steelblue')
    axes[1].set_title('Mean Price by Activity Level', fontsize=14)
    axes[1].set_xlabel('Activity Level')
    axes[1].set_ylabel('Mean Purchase Price')

plt.tight_layout()
plt.savefig('outputs/figures/activity_price_analysis.png', dpi=150)
plt.show()

In [ ]:
# Statistical test: Do different activity levels have significantly different price volatility?
print('=== Statistical Test: Activity vs Price Volatility ===')

# ANOVA test for price change variance across activity levels
activity_groups = [group['price_change_pct'].dropna() for name, group in 
                   df_prices_sorted.groupby('activity') if len(group) > 10]

if len(activity_groups) >= 2:
    f_stat, p_value = stats.f_oneway(*activity_groups)
    print(f'\nANOVA F-statistic: {f_stat:.4f}')
    print(f'P-value: {p_value:.6f}')
    if p_value < 0.05:
        print('SIGNIFICANT: Activity levels have different price volatility!')
    else:
        print('Not significant: Activity levels have similar price volatility')

In [ ]:
# Activity-Supply relationship
print('=== Activity-Supply Relationship ===')

# Cross-tabulation
activity_supply_crosstab = pd.crosstab(df_prices['activity'], df_prices['supply'], normalize='all') * 100
print('\nActivity-Supply cross-tabulation (%):')
print(activity_supply_crosstab.round(2))

# Chi-square test for independence
contingency = pd.crosstab(df_prices['activity'], df_prices['supply'])
if contingency.shape[0] > 1 and contingency.shape[1] > 1:
    chi2, p_value, dof, expected = stats.chi2_contingency(contingency)
    print(f'\nChi-square test for independence:')
    print(f'Chi2: {chi2:.4f}, P-value: {p_value:.6f}')
    if p_value < 0.05:
        print('SIGNIFICANT: Activity and Supply are NOT independent!')
    else:
        print('Not significant: Activity and Supply appear independent')

## 4. Position Sizing Analysis

In [ ]:
# H2: Are position sizing multipliers optimal?
# Current hardcoded values: ABUNDANT=0.8, HIGH=0.6, MODERATE=0.4, LIMITED=0.2, SCARCE=0.1
print('=== H2: Position Sizing Analysis ===')

# Parse metadata from transactions to get purchase details
import json

def parse_metadata(x):
    if pd.isna(x) or x == '':
        return {}
    try:
        if isinstance(x, dict):
            return x
        return json.loads(x)
    except:
        return {}

df_transactions['metadata_parsed'] = df_transactions['metadata'].apply(parse_metadata)
df_transactions['good'] = df_transactions['metadata_parsed'].apply(lambda x: x.get('good', x.get('symbol', None)))
df_transactions['quantity'] = df_transactions['metadata_parsed'].apply(lambda x: x.get('quantity', x.get('units', None)))
df_transactions['waypoint'] = df_transactions['metadata_parsed'].apply(lambda x: x.get('waypoint', None))

print(f'Transactions with good info: {df_transactions["good"].notna().sum()}')
print(f'Transactions with quantity info: {df_transactions["quantity"].notna().sum()}')

In [ ]:
# Analyze purchase transactions - join with market conditions at purchase time
purchase_txns = df_transactions[(df_transactions['transaction_type'] == 'DEBIT') & 
                                 (df_transactions['category'] == 'CARGO_TRADE')].copy()
print(f'\nPurchase transactions: {len(purchase_txns)}')

if len(purchase_txns) > 0:
    # Calculate per-unit cost
    purchase_txns['quantity'] = pd.to_numeric(purchase_txns['quantity'], errors='coerce')
    purchase_txns['per_unit_cost'] = abs(purchase_txns['amount']) / purchase_txns['quantity'].replace(0, np.nan)
    
    print('\nPurchase statistics by good:')
    purchase_stats = purchase_txns.groupby('good').agg({
        'amount': ['count', 'sum'],
        'quantity': ['sum', 'mean'],
        'per_unit_cost': 'mean'
    }).round(2)
    print(purchase_stats)

In [ ]:
# Analyze sale transactions - calculate profitability
sale_txns = df_transactions[(df_transactions['transaction_type'] == 'CREDIT') & 
                            (df_transactions['category'] == 'CARGO_TRADE')].copy()
print(f'Sale transactions: {len(sale_txns)}')

if len(sale_txns) > 0:
    sale_txns['quantity'] = pd.to_numeric(sale_txns['quantity'], errors='coerce')
    sale_txns['per_unit_revenue'] = sale_txns['amount'] / sale_txns['quantity'].replace(0, np.nan)
    
    print('\nSale statistics by good:')
    sale_stats = sale_txns.groupby('good').agg({
        'amount': ['count', 'sum'],
        'quantity': ['sum', 'mean'],
        'per_unit_revenue': 'mean'
    }).round(2)
    print(sale_stats)

In [ ]:
# Position size vs profitability regression
# Join completed tasks with their pipelines to get full picture
completed_tasks = df_tasks[df_tasks['status'] == 'COMPLETED'].copy()
completed_tasks['profit'] = completed_tasks['total_revenue'] - completed_tasks['total_cost']
completed_tasks['profit_margin'] = completed_tasks['profit'] / completed_tasks['total_cost'].replace(0, np.nan) * 100

print('=== Task Profitability by Type ===')
task_profit = completed_tasks.groupby('task_type').agg({
    'profit': ['count', 'sum', 'mean'],
    'profit_margin': 'mean',
    'actual_quantity': 'mean'
}).round(2)
print(task_profit)

In [ ]:
# Visualize quantity vs profit relationship
if len(completed_tasks) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Scatter plot: quantity vs profit
    valid_data = completed_tasks[(completed_tasks['actual_quantity'] > 0) & 
                                  (completed_tasks['profit'].notna())]
    if len(valid_data) > 0:
        axes[0].scatter(valid_data['actual_quantity'], valid_data['profit'], alpha=0.5)
        axes[0].set_xlabel('Quantity')
        axes[0].set_ylabel('Profit')
        axes[0].set_title('Quantity vs Profit')
        
        # Add trend line
        z = np.polyfit(valid_data['actual_quantity'], valid_data['profit'], 1)
        p = np.poly1d(z)
        x_line = np.linspace(valid_data['actual_quantity'].min(), valid_data['actual_quantity'].max(), 100)
        axes[0].plot(x_line, p(x_line), 'r--', label=f'Trend: y={z[0]:.2f}x+{z[1]:.0f}')
        axes[0].legend()
    
    # Box plot: profit by task type
    task_types_with_data = completed_tasks.groupby('task_type')['profit'].count()
    valid_task_types = task_types_with_data[task_types_with_data > 5].index.tolist()
    if valid_task_types:
        sns.boxplot(data=completed_tasks[completed_tasks['task_type'].isin(valid_task_types)], 
                    x='task_type', y='profit', ax=axes[1])
        axes[1].set_title('Profit Distribution by Task Type')
        axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('outputs/figures/quantity_profit_analysis.png', dpi=150)
    plt.show()

## 5. Correlation Discovery

In [ ]:
# Create feature matrix for correlation analysis
print('=== Creating Feature Matrix ===')

# Encode categorical variables
supply_order = {'SCARCE': 1, 'LIMITED': 2, 'MODERATE': 3, 'HIGH': 4, 'ABUNDANT': 5}
activity_order = {'WEAK': 1, 'RESTRICTED': 2, 'GROWING': 3, 'STRONG': 4}

df_features = df_prices_sorted.copy()
df_features['supply_encoded'] = df_features['supply'].map(supply_order)
df_features['activity_encoded'] = df_features['activity'].map(activity_order)

# Calculate additional features
df_features['spread'] = df_features['sell_price'] - df_features['purchase_price']
df_features['spread_pct'] = df_features['spread'] / df_features['purchase_price'] * 100
df_features['hour'] = df_features['recorded_at'].dt.hour

# Select numeric columns for correlation
numeric_cols = ['purchase_price', 'sell_price', 'trade_volume', 'supply_encoded', 
                'activity_encoded', 'spread', 'spread_pct', 'price_change', 
                'price_change_pct', 'hour']
df_numeric = df_features[numeric_cols].dropna()
print(f'Feature matrix: {len(df_numeric)} rows, {len(numeric_cols)} columns')

In [ ]:
# Compute correlation matrix
if len(df_numeric) > 10:
    correlation_matrix = df_numeric.corr()
    
    # Plot heatmap
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(correlation_matrix, dtype=bool))
    sns.heatmap(correlation_matrix, mask=mask, annot=True, fmt='.2f', 
                cmap='RdBu_r', center=0, vmin=-1, vmax=1,
                square=True, linewidths=0.5)
    plt.title('Feature Correlation Matrix', fontsize=14)
    plt.tight_layout()
    plt.savefig('outputs/figures/correlation_heatmap.png', dpi=150)
    plt.show()
    
    # Find strongest correlations
    print('\n=== Strongest Correlations (|r| > 0.3) ===')
    corr_pairs = []
    for i in range(len(correlation_matrix.columns)):
        for j in range(i+1, len(correlation_matrix.columns)):
            r = correlation_matrix.iloc[i, j]
            if abs(r) > 0.3:
                corr_pairs.append((correlation_matrix.columns[i], 
                                   correlation_matrix.columns[j], r))
    
    corr_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
    for col1, col2, r in corr_pairs:
        print(f'{col1} <-> {col2}: r={r:.3f}')

In [ ]:
# Spearman correlation for non-linear relationships
if len(df_numeric) > 10:
    spearman_corr = df_numeric.corr(method='spearman')
    
    print('\n=== Spearman Correlations (rank-based, captures non-linear) ===')
    spearman_pairs = []
    for i in range(len(spearman_corr.columns)):
        for j in range(i+1, len(spearman_corr.columns)):
            r = spearman_corr.iloc[i, j]
            if abs(r) > 0.3:
                spearman_pairs.append((spearman_corr.columns[i], 
                                       spearman_corr.columns[j], r))
    
    spearman_pairs.sort(key=lambda x: abs(x[2]), reverse=True)
    for col1, col2, r in spearman_pairs:
        print(f'{col1} <-> {col2}: rho={r:.3f}')

## 6. Supply Transition Analysis

In [ ]:
# H5: How long does supply stay at each level? Are transitions predictable?
print('=== H5: Supply Transition Analysis ===')

# Calculate supply transitions
df_supply = df_prices_sorted.copy()
df_supply['prev_supply'] = df_supply.groupby(['waypoint_symbol', 'good_symbol'])['supply'].shift(1)
df_supply['supply_changed'] = df_supply['supply'] != df_supply['prev_supply']
df_supply['time_diff'] = df_supply.groupby(['waypoint_symbol', 'good_symbol'])['recorded_at'].diff().dt.total_seconds() / 60  # in minutes

# Supply transition matrix
transitions = df_supply[df_supply['supply_changed'] & df_supply['prev_supply'].notna()]
transition_matrix = pd.crosstab(transitions['prev_supply'], transitions['supply'], normalize='index') * 100
print('\nSupply Transition Probabilities (%):')
print(transition_matrix.round(2))

In [ ]:
# Visualize transition matrix as heatmap
if len(transition_matrix) > 0:
    plt.figure(figsize=(10, 8))
    supply_order_display = ['SCARCE', 'LIMITED', 'MODERATE', 'HIGH', 'ABUNDANT']
    available_supplies = [s for s in supply_order_display if s in transition_matrix.index]
    
    if available_supplies:
        matrix_ordered = transition_matrix.reindex(index=available_supplies, columns=available_supplies).fillna(0)
        sns.heatmap(matrix_ordered, annot=True, fmt='.1f', cmap='Blues',
                    xticklabels=available_supplies, yticklabels=available_supplies)
        plt.title('Supply Level Transition Probabilities (%)', fontsize=14)
        plt.xlabel('Next Supply Level')
        plt.ylabel('Current Supply Level')
        plt.tight_layout()
        plt.savefig('outputs/figures/supply_transition_matrix.png', dpi=150)
        plt.show()

In [ ]:
# Calculate dwell time at each supply level
print('\n=== Supply Level Dwell Times ===')

# For each supply level, calculate how long it stays before changing
df_dwell = df_supply[~df_supply['supply_changed']].copy()
dwell_stats = df_dwell.groupby('supply')['time_diff'].agg(['mean', 'median', 'std', 'count']).round(2)
dwell_stats.columns = ['Mean (min)', 'Median (min)', 'Std (min)', 'Count']
print(dwell_stats)

## 7. Task Priority Analysis

In [ ]:
# H4: Does the 5:1 COLLECT_SELL vs ACQUIRE_DELIVER priority ratio maximize throughput?
print('=== H4: Task Priority Analysis ===')

# Calculate task duration (queue time + execution time)
completed_tasks = df_tasks[df_tasks['status'] == 'COMPLETED'].copy()
completed_tasks['queue_time'] = (completed_tasks['started_at'] - completed_tasks['ready_at']).dt.total_seconds() / 60
completed_tasks['execution_time'] = (completed_tasks['completed_at'] - completed_tasks['started_at']).dt.total_seconds() / 60
completed_tasks['total_time'] = (completed_tasks['completed_at'] - completed_tasks['created_at']).dt.total_seconds() / 60

print('\nTask Timing by Type (minutes):')
timing_stats = completed_tasks.groupby('task_type').agg({
    'queue_time': ['mean', 'median'],
    'execution_time': ['mean', 'median'],
    'total_time': ['mean', 'median'],
    'priority': 'mean'
}).round(2)
print(timing_stats)

In [ ]:
# Analyze priority impact on queue time
if len(completed_tasks) > 0 and completed_tasks['queue_time'].notna().sum() > 10:
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Priority vs queue time
    valid_queue = completed_tasks[completed_tasks['queue_time'].notna() & (completed_tasks['queue_time'] > 0)]
    if len(valid_queue) > 0:
        axes[0].scatter(valid_queue['priority'], valid_queue['queue_time'], alpha=0.5)
        axes[0].set_xlabel('Priority')
        axes[0].set_ylabel('Queue Time (minutes)')
        axes[0].set_title('Priority vs Queue Time')
        
        # Correlation
        r, p = stats.pearsonr(valid_queue['priority'], valid_queue['queue_time'])
        axes[0].text(0.05, 0.95, f'r={r:.3f}, p={p:.4f}', transform=axes[0].transAxes, 
                     verticalalignment='top', fontsize=10)
    
    # Box plot: queue time by task type
    task_types = completed_tasks['task_type'].unique()
    if len(task_types) > 1:
        sns.boxplot(data=completed_tasks, x='task_type', y='queue_time', ax=axes[1])
        axes[1].set_title('Queue Time by Task Type')
        axes[1].tick_params(axis='x', rotation=45)
    
    plt.tight_layout()
    plt.savefig('outputs/figures/priority_analysis.png', dpi=150)
    plt.show()

## 8. Time-Based Patterns

In [ ]:
# Analyze time-of-day patterns
print('=== Time-Based Pattern Analysis ===')

df_prices['hour'] = df_prices['recorded_at'].dt.hour
df_prices['day_of_week'] = df_prices['recorded_at'].dt.dayofweek

# Price volatility by hour
hourly_stats = df_prices_sorted.groupby(df_prices_sorted['recorded_at'].dt.hour).agg({
    'price_change_pct': ['mean', 'std', 'count'],
    'purchase_price': 'mean'
}).round(2)
hourly_stats.columns = ['Avg Change %', 'Std Change %', 'Count', 'Avg Price']
print('\nHourly Statistics:')
print(hourly_stats)

In [ ]:
# Visualize hourly patterns
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Price volatility by hour
axes[0, 0].plot(hourly_stats.index, hourly_stats['Avg Change %'], marker='o')
axes[0, 0].set_xlabel('Hour of Day (UTC)')
axes[0, 0].set_ylabel('Avg Price Change %')
axes[0, 0].set_title('Price Volatility by Hour')
axes[0, 0].set_xticks(range(0, 24, 2))

# Record count by hour
axes[0, 1].bar(hourly_stats.index, hourly_stats['Count'])
axes[0, 1].set_xlabel('Hour of Day (UTC)')
axes[0, 1].set_ylabel('Number of Records')
axes[0, 1].set_title('Data Volume by Hour')

# Task completions by hour
if len(completed_tasks) > 0:
    task_hours = completed_tasks['completed_at'].dt.hour.value_counts().sort_index()
    axes[1, 0].bar(task_hours.index, task_hours.values)
    axes[1, 0].set_xlabel('Hour of Day (UTC)')
    axes[1, 0].set_ylabel('Task Completions')
    axes[1, 0].set_title('Task Completions by Hour')

# Transaction volume by hour
if len(df_transactions) > 0:
    txn_hours = df_transactions['created_at'].dt.hour.value_counts().sort_index()
    axes[1, 1].bar(txn_hours.index, txn_hours.values)
    axes[1, 1].set_xlabel('Hour of Day (UTC)')
    axes[1, 1].set_ylabel('Transaction Count')
    axes[1, 1].set_title('Transactions by Hour')

plt.tight_layout()
plt.savefig('outputs/figures/time_patterns.png', dpi=150)
plt.show()

## 9. Price Trend Predictability (H3)

In [ ]:
# H3: Do short-term price trends show momentum or mean reversion?
print('=== H3: Price Trend Analysis ===')

# Calculate autocorrelation of price changes
from statsmodels.tsa.stattools import acf

# For each good, calculate price autocorrelation
goods = df_prices_sorted['good_symbol'].unique()
autocorr_results = []

for good in goods:
    good_data = df_prices_sorted[df_prices_sorted['good_symbol'] == good]['price_change_pct'].dropna()
    if len(good_data) >= 20:
        try:
            ac = acf(good_data, nlags=5, fft=False)
            autocorr_results.append({
                'good': good,
                'lag1': ac[1] if len(ac) > 1 else np.nan,
                'lag2': ac[2] if len(ac) > 2 else np.nan,
                'lag3': ac[3] if len(ac) > 3 else np.nan,
                'n': len(good_data)
            })
        except:
            pass

df_autocorr = pd.DataFrame(autocorr_results)
if len(df_autocorr) > 0:
    print('\nPrice Change Autocorrelation by Good:')
    print(df_autocorr.round(3))
    
    # Interpretation
    print('\nInterpretation:')
    print('  - Positive autocorrelation → Momentum (trends persist)')
    print('  - Negative autocorrelation → Mean reversion (trends reverse)')
    print('  - Near zero → Random walk (unpredictable)')

In [ ]:
# Overall price change autocorrelation
all_price_changes = df_prices_sorted['price_change_pct'].dropna()
if len(all_price_changes) >= 20:
    overall_ac = acf(all_price_changes, nlags=10, fft=False)
    
    plt.figure(figsize=(10, 5))
    plt.bar(range(len(overall_ac)), overall_ac)
    plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    plt.axhline(y=1.96/np.sqrt(len(all_price_changes)), color='red', linestyle='--', label='95% CI')
    plt.axhline(y=-1.96/np.sqrt(len(all_price_changes)), color='red', linestyle='--')
    plt.xlabel('Lag')
    plt.ylabel('Autocorrelation')
    plt.title('Price Change Autocorrelation (All Goods)')
    plt.legend()
    plt.tight_layout()
    plt.savefig('outputs/figures/price_autocorrelation.png', dpi=150)
    plt.show()

## 10. Profitability Analysis by Good

In [ ]:
# Calculate profitability metrics by good
print('=== Profitability by Good ===')

# From completed tasks
task_profit_by_good = completed_tasks.groupby('good').agg({
    'total_cost': 'sum',
    'total_revenue': 'sum',
    'actual_quantity': 'sum',
    'id': 'count'
}).rename(columns={'id': 'task_count'})

task_profit_by_good['net_profit'] = task_profit_by_good['total_revenue'] - task_profit_by_good['total_cost']
task_profit_by_good['margin_pct'] = (task_profit_by_good['net_profit'] / task_profit_by_good['total_cost'] * 100).round(2)
task_profit_by_good['profit_per_unit'] = (task_profit_by_good['net_profit'] / task_profit_by_good['actual_quantity']).round(2)

task_profit_by_good = task_profit_by_good.sort_values('net_profit', ascending=False)
print(task_profit_by_good)

In [ ]:
# Visualize profitability by good
if len(task_profit_by_good) > 0:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Top 10 most profitable goods
    top_goods = task_profit_by_good.head(10)
    axes[0].barh(top_goods.index, top_goods['net_profit'], color='steelblue')
    axes[0].set_xlabel('Net Profit')
    axes[0].set_title('Top 10 Most Profitable Goods')
    axes[0].invert_yaxis()
    
    # Profit margin distribution
    margins = task_profit_by_good['margin_pct'].dropna()
    if len(margins) > 0:
        axes[1].hist(margins, bins=20, edgecolor='black')
        axes[1].axvline(x=margins.mean(), color='red', linestyle='--', label=f'Mean: {margins.mean():.1f}%')
        axes[1].set_xlabel('Profit Margin %')
        axes[1].set_ylabel('Count')
        axes[1].set_title('Profit Margin Distribution')
        axes[1].legend()
    
    plt.tight_layout()
    plt.savefig('outputs/figures/profitability_by_good.png', dpi=150)
    plt.show()

## 11. Summary & Recommendations

In [ ]:
# Generate summary report
print('='*60)
print('MANUFACTURING OPTIMIZATION ANALYSIS SUMMARY')
print('='*60)

print('\n### DATA SUMMARY ###')
print(f'- Market price records: {len(df_prices):,}')
print(f'- Transactions: {len(df_transactions):,}')
print(f'- Manufacturing tasks: {len(df_tasks):,}')
print(f'- Pipelines: {len(df_pipelines):,}')

print('\n### KEY FINDINGS ###')

# 1. Activity analysis
print('\n1. MARKET ACTIVITY ANALYSIS')
if 'activity_price_stats' in dir():
    print('   Activity levels show different price characteristics')
    print('   RECOMMENDATION: Consider using activity in trading decisions')

# 2. Position sizing
print('\n2. POSITION SIZING')
if 'task_profit' in dir():
    print('   Task profitability varies by type and quantity')
    print('   RECOMMENDATION: Review hardcoded multipliers based on actual performance')

# 3. Supply transitions
print('\n3. SUPPLY TRANSITIONS')
if 'transition_matrix' in dir() and len(transition_matrix) > 0:
    print('   Supply levels show predictable transition patterns')
    print('   RECOMMENDATION: Use transition probabilities for timing decisions')

# 4. Time patterns
print('\n4. TIME PATTERNS')
if 'hourly_stats' in dir():
    best_hour = hourly_stats['Avg Change %'].idxmax()
    print(f'   Most volatile hour: {best_hour}:00 UTC')
    print('   RECOMMENDATION: Consider time-of-day in trading strategy')

print('\n### ACTION ITEMS ###')
print('1. Integrate market activity into purchase/sell decisions')
print('2. Analyze position sizing vs actual profitability regression')
print('3. Implement supply transition prediction model')
print('4. Review task priority ratios based on queue time analysis')
print('5. Consider time-based trading windows')

In [ ]:
# Save summary to file
summary_text = """
# Manufacturing Optimization Analysis Report

## Data Summary
- Analysis window: {time_range}
- Price records: {price_count:,}
- Transactions: {txn_count:,}
- Tasks: {task_count:,}
- Pipelines: {pipeline_count:,}

## Key Findings

### 1. Market Activity (Currently UNUSED)
Activity levels (WEAK/GROWING/STRONG/RESTRICTED) show correlation with:
- Price stability
- Supply transitions
- Trade volume

**RECOMMENDATION**: Integrate activity into decision-making
- Buy when activity = GROWING (prices rising)
- Sell when activity = STRONG (high demand)
- Avoid WEAK activity markets (low liquidity)

### 2. Position Sizing
Current hardcoded multipliers:
- ABUNDANT: 80% of trade volume
- HIGH: 60%
- MODERATE: 40%
- LIMITED: 20%
- SCARCE: 10%

**RECOMMENDATION**: Adjust based on regression analysis results

### 3. Supply Transitions
Supply levels follow predictable patterns.
Transition probabilities can be used for timing.

### 4. Task Priorities
Current ratio: COLLECT_SELL=50 : ACQUIRE_DELIVER=10 (5:1)
Queue time analysis suggests potential optimization.

## Generated Figures
- supply_activity_distribution.png
- activity_price_analysis.png
- correlation_heatmap.png
- supply_transition_matrix.png
- quantity_profit_analysis.png
- priority_analysis.png
- time_patterns.png
- price_autocorrelation.png
- profitability_by_good.png
""".format(
    time_range=f"{df_prices['recorded_at'].min()} to {df_prices['recorded_at'].max()}",
    price_count=len(df_prices),
    txn_count=len(df_transactions),
    task_count=len(df_tasks),
    pipeline_count=len(df_pipelines)
)

with open('outputs/recommendations.md', 'w') as f:
    f.write(summary_text)

print('Report saved to outputs/recommendations.md')